# Notebook setup

In [ ]:
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba
import bw2io as bi
import numpy as np
from scipy import sparse

This notebook assumes you already have a project with ecoinvent.

In [ ]:
bd.projects.set_current("USEEIO example")

In [ ]:
bi.useeio11()

In [ ]:
useeio = bd.Database("US EEIO 1.1")

In [ ]:
bd.methods

# Contribution analysis

Before we dive in to `bw2analyzer`, we can actually do the basic contribution analysis easily by ourselves. All we need is to sum one axis of the `lca.characterized_inventory` matrix, sort the values and indices, and then retrieve the metadata for the indices with the highest values.

In [ ]:
our_act = useeio.random()

while our_act['type'] != 'product':
    our_act = useeio.random()

our_fu = {our_act: 1}

In [ ]:
our_act

In [ ]:
lca = bc.LCA(our_fu, ('Impact Potential', 'GCC'))
lca.lci()
lca.lcia()
lca.score

Let's look at activities. In this case, we need to sum the rows (stressors or biosphere flows), which are the first axis. As Python is 0-indexed, we use `axis=0`:

In [ ]:
lca.characterized_inventory.shape

In [ ]:
by_activity = np.array(lca.characterized_inventory.sum(axis=0))
by_activity.shape

We then need to sort the values. As some may be large, but negative, we sort by the absolute value.

There are tricks to do this very quickly in numpy, but we can also be simple:

In [ ]:
sorted_values = sorted([(abs(value), value, index) for index, value in enumerate(by_activity[0, :])], reverse=True)

In [ ]:
sorted_values[:5]

Then we need to relate these **matrix** indices to something useful. We do this by going backwards, not from database IDs to matrix indices, but matrix indices to database IDs:

In [ ]:
for _, value, index in sorted_values[:5]:
    print(value, bd.get_activity(lca.dicts.activity.reversed[index]))

We can also get this information with `bw2analyzer`:

In [ ]:
ba.ContributionAnalysis().annotated_top_processes(lca)[:5]

The returned format here is `(LCIA score, amount in supply array, activity metadata)`.

Similary, we can look at the emissions:

In [ ]:
ba.ContributionAnalysis().annotated_top_emissions(lca)[:5]

In [ ]:
%debug

# Miscellaneous functions

## Graphing the technosphere matrix

In [ ]:
from bw2analyzer.matrix_grapher import SparseMatrixGrapher 

In [ ]:
smg = SparseMatrixGrapher(lca.technosphere_matrix).ordered_graph(width=8, height=8)

This isn't so interesting for an IO table, but gets cooler for process databases with lots of empty space. We can simulate that:

In [ ]:
def shrink_sparse_matrix(matrix, fill_rate=0.025):
    m = matrix.tocoo()
    indices = np.random.randint(0, m.nnz, size=round(m.nnz * fill_rate))
    return sparse.csr_matrix((m.data[indices], (m.row[indices], m.col[indices])), shape=m.shape)

In [ ]:
smg = SparseMatrixGrapher(shrink_sparse_matrix(lca.technosphere_matrix, 0.05)).ordered_graph(width=8, height=8)

## Economic analysis

Do the rich activities have all the impact in this world?

In [ ]:
from bw2analyzer.econ import gini_coefficient, herfindahl_index, concentration_ratio, theil_index

In [ ]:
income = lca.characterized_inventory.data

In [ ]:
gini_coefficient(income), herfindahl_index(income), concentration_ratio(income), theil_index(income)

# Traversing the supply chain

There is an industrial-strength class in `bw2calc` called `GraphTraversal`, but for quick analysis we can use the following:

In [ ]:
ba.print_recursive_supply_chain(our_act, max_level=1)

Unfortunately, this notebook uses only free data, and the USEEIO uses `activities` and `products`, and this function assumes only activities so it isn't so helpful in this case...

In the above function, you can play with the following default arguments:

* amount=1,
* max_level=2,
* cutoff=0,
* file_obj=None,
* tab_character="  ",
* level=0,

Similarly, we can traverse the *characterized* inventory.

In [ ]:
ba.print_recursive_calculation(our_act, ('Impact Potential', 'GCC'))

Same thing here...